In [ ]:
from sqlalchemy import text
import requests
import random
from datetime import datetime, timedelta
import time
import math
from collections import defaultdict
import pandas as pd
from config import db_url

In [ ]:
class Ruang:
    def __init__(self, nama, tipe_ruang):
        self.nama = nama
        self.tipe_ruang = tipe_ruang
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))
    
    def __repr__(self):
        return f"Ruang(nama={self.nama}, tipe_ruang={self.tipe_ruang})"
class Dosen:
    def __init__(self, nama):
        self.nama = nama
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))
    def __repr__(self):
        return f"Dosen(nama={self.nama})"

class Matakuliah:
    def __init__(self, matkul, dosen, sks, kelas, id_perkuliahan, id_semester, semester,kategori,prodi,status = None ,kode_mk=None, kode_pasangan=None): #tambah id_perkuliahan
        self.id_perkuliahan = id_perkuliahan
        self.matkul = matkul
        self.dosen = dosen
        self.sks = sks
        self.status = status
        self.kelas = kelas
        self.id_semester = id_semester
        self.semester = semester
        self.kategori = kategori
        self.prodi=prodi
        self.kode_mk = kode_mk  # Tambahan field
        self.kode_pasangan = kode_pasangan  # Tambahan field
        self.ruang_needed = self.set_ruang(kategori,status)

    def __repr__(self):
        return (f"matkul(matkul={self.matkul}, dosen={self.dosen}, sks={self.sks}, status={self.status})")
    
    def set_ruang(self, kategori, status = None):
        if kategori == "Teori":
            return [status] if status else ["Kelas"]
        elif kategori == "Praktikum":
            return ["Lab"]
        elif kategori == "Gabungan":
            return [status] if status else ["Kelas", "Lab"]
        else:
            return []

In [ ]:
class PenjadwalanSA:
    def __init__(self, initial_temperature=15000, cooling_rate=0.9999, max_iterations=900000, id_generate='G2406-07'):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.id_generate = id_generate
        self.engine = db_url
        
        # Data penjadwalan
        self.daftar_dosen = []
        self.daftar_ruang = []
        self.daftar_matkul = []
        self.daftar_hari = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
        

        # Slot waktu 
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = self.generate_slot_waktu()
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

        self.baca_datamk()
        self.baca_dataruang()

    def baca_datamk(self):
        query = text("""
        SELECT tb_rombel.nama_kelas AS kelas,
            tb_matakuliah.nama_matakuliah AS matakuliah,
            tb_dosen.nama_dosen AS dosen,
            tb_matakuliah.sks AS sks,
            tb_matakuliah.status AS status,
            tb_matakuliah.kategori AS kategori,
            tb_matakuliah.kode_matakuliah AS kode_mk,  
            tb_matakuliah.kode_pasangan AS kode_pasangan,
            id_perkuliahan,
            id_semester,
            tb_matakuliah.nama_semester AS semester,
            tb_prodi.nama_prodi AS prodi
        FROM tb_perkuliahan                    
        JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
        JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
        JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
        JOIN tb_generate ON tb_generate.id_generate = tb_rombel.id_generate
        JOIN tb_prodi ON tb_perkuliahan.kode_prodi = tb_prodi.kode_prodi
        AND tb_generate.id_generate = tb_matakuliah.id_generate 
        AND tb_generate.id_generate = tb_dosen.id_generate
        AND tb_generate.id_generate = tb_prodi.id_generate
        WHERE tb_generate.id_generate = 'G2406-07'
        """)
        with self.engine.connect() as connection:
            df_matkul = pd.read_sql_query(query, connection, params={"id_generate": self.id_generate})

        # Iterasi per baris untuk menambahkan data
        for _, row in df_matkul.iterrows():
            dosen_obj = self.tambah_dosen(row['dosen'])
            self.tambah_matkul(
                row['matakuliah'], dosen_obj, row['sks'], row['kelas'], 
                row['status'], row['id_perkuliahan'], row['id_semester'], 
                row['semester'], row['kategori'], row['prodi'], 
                row['kode_mk'], row['kode_pasangan']
            )

    def baca_datadosen(self):
        query = text("""
            SELECT nama_dosen AS dosen
            FROM tb_dosen
            JOIN tb_generate ON tb_dosen.id_generate = tb_generate.id_generate
            WHERE tb_generate.status = 'belum' 
            AND tb_generate.id_generate = 'G2406-07'
        """)

        with self.engine.connect() as connection:
            df_dosen = pd.read_sql_query(query, connection,params={"id_generate": self.id_generate})
        for _, row in df_dosen.iterrows():
            self.tambah_dosen(row['dosen'])

    def baca_dataruang(self):
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        WHERE tb_generate.status = 'belum' 
        AND tb_generate.id_generate = 'G2406-07'
        """)
        with self.engine.connect() as connection:
            df_ruang = pd.read_sql_query(query, connection,params={"id_generate": self.id_generate})

        # Iterasi per baris untuk menambahkan data
        for _, row in df_ruang.iterrows():
            self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
            
        
    def tambah_dosen(self, nama):
        for d in self.daftar_dosen:
            if d.nama == nama:
                return d
        dosen = Dosen(nama)
        self.daftar_dosen.append(dosen)
        return dosen
    
    def tambah_ruang(self, nama, tiperuang):
        ruang = Ruang(nama,tiperuang)
        self.daftar_ruang.append(ruang)
        return ruang
    
    def tambah_matkul(self, matkul, dosen, sks, kelas, status, id_perkuliahan, id_semester, semester, kategori, prodi, kode_mk=None, kode_pasangan=None):
        matkul = Matakuliah(matkul, dosen, sks, kelas, id_perkuliahan, id_semester, semester, kategori, prodi, status, kode_mk, kode_pasangan)
        self.daftar_matkul.append(matkul)
        return matkul

    def generate_slot_waktu(self):
        daftar_slot = []
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    # Slot tumpang tindih dengan istirahat
                    waktu_mulai = istirahat_selesai  # lanjut setelah istirahat
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                daftar_slot.append((
                    waktu_mulai.strftime("%H:%M"),
                    waktu_selesai.strftime("%H:%M")
                ))
                waktu_mulai = waktu_selesai
        return daftar_slot

    def jam_sks(self, sks, kategori):
        if kategori == "Teori":
            menit_total = sks * 50
        elif kategori == "Praktikum":
            menit_total = sks * 150
        elif kategori == "Gabungan":
            menit_total = 350 
        else:
            menit_total = 0
        return math.ceil(menit_total / 50)
    
    def get_ruang_valid(self,matkul):
        ruang_valid = [ruang for ruang in self.daftar_ruang if not matkul.ruang_needed or any(t in ruang.tipe_ruang for t in matkul.ruang_needed)]
        return random.choice(ruang_valid) if ruang_valid else random.choice(self.daftar_ruang)

    def find_paired_course(self, matkul):
        if not matkul.kode_pasangan:
            return None
        
        for mk in self.daftar_matkul:
            if mk.kode_mk == matkul.kode_pasangan:
                return mk
        return None
    
    def is_paired_courses(self, mk1, mk2):
        # Cek apakah mk1 (Teori) memiliki pasangan mk2 (Praktikum)
        if mk1.kode_pasangan and mk1.kode_pasangan == mk2.kode_mk:
            return True
        # Cek apakah mk2 (Teori) memiliki pasangan mk1 (Praktikum)
        if mk2.kode_pasangan and mk2.kode_pasangan == mk1.kode_mk:
            return True
        return False
    
    def find_praktikum_from_teori(self, matkul_teori):
        """Mencari mata kuliah praktikum berdasarkan mata kuliah teori"""
        if not matkul_teori.kode_pasangan:
            return None
        
        for mk in self.daftar_matkul:
            if mk.kode_mk == matkul_teori.kode_pasangan:
                return mk
        return None

    # Tambahkan method untuk mencari mata kuliah teori berdasarkan mata kuliah praktikum
    def find_teori_from_praktikum(self, matkul_praktikum):
        """Mencari mata kuliah teori yang memiliki pasangan praktikum ini"""
        for mk in self.daftar_matkul:
            if mk.kode_pasangan == matkul_praktikum.kode_mk:
                return mk
        return None

    def anneal(self):
        current_solution = self.solusi_awal()
        current_energy = self.calculate_energy(current_solution)
        best_solution = current_solution
        best_energy = current_energy
        temperature = self.initial_temperature

        for i in range(self.max_iterations):
            neighbor_solution = self.get_neighbor(current_solution)
            neighbor_energy = self.calculate_energy(neighbor_solution)

            if self.accept_probability(current_energy, neighbor_energy, temperature) > random.random():
                current_solution = neighbor_solution
                current_energy = neighbor_energy

            if current_energy < best_energy:
                best_solution = current_solution
                best_energy = current_energy

            temperature *= self.cooling_rate
            if i % 10000 == 0:
                print(f"Iterasi {i} | Skor saat ini: {current_energy} | Skor terbaik: {best_energy} | Temperatur: {temperature:.4f}")

        self.apply_solution(best_solution)
        self.best_solution = best_solution 


    def solusi_awal(self):
        solution = []
        scheduled_courses = set()  # Track kode_mk mata kuliah yang sudah dijadwalkan
        
        for matkul in self.daftar_matkul:
            if matkul.kode_mk in scheduled_courses:
                continue  # Skip jika sudah dijadwalkan sebagai pasangan
                
            # Jika mata kuliah teori memiliki pasangan praktikum
            if matkul.kode_pasangan:
                praktikum_course = self.find_praktikum_from_teori(matkul)
                
                if praktikum_course and praktikum_course.kode_mk not in scheduled_courses:
                    # Jadwalkan mata kuliah berpasangan berurutan (teori dulu, lalu praktikum)
                    durasi_teori = self.jam_sks(matkul.sks, matkul.kategori)
                    durasi_praktikum = self.jam_sks(praktikum_course.sks, praktikum_course.kategori)
                    total_durasi = durasi_teori + durasi_praktikum
                    
                    max_jam_mulai = len(self.daftar_slot) - total_durasi
                    
                    if max_jam_mulai > 0:
                        ruang_teori = self.get_ruang_valid(matkul)
                        ruang_praktikum = self.get_ruang_valid(praktikum_course)
                        hari = random.choice(self.daftar_hari)
                        jam_mulai_teori = random.randint(0, max_jam_mulai)
                        jam_mulai_praktikum = jam_mulai_teori + durasi_teori
                        
                        solution.append((matkul, ruang_teori, hari, jam_mulai_teori))
                        solution.append((praktikum_course, ruang_praktikum, hari, jam_mulai_praktikum))
                        
                        scheduled_courses.add(matkul.kode_mk)
                        scheduled_courses.add(praktikum_course.kode_mk)
                    else:
                        print(f"Mata kuliah berpasangan {matkul.matkul} dan {praktikum_course.matkul} tidak muat dalam satu hari")
                        # Jadwalkan teori saja terlebih dahulu
                        ruang = self.get_ruang_valid(matkul)
                        hari = random.choice(self.daftar_hari)
                        jam_mulai = random.randint(0, len(self.daftar_slot) - durasi_teori)
                        solution.append((matkul, ruang, hari, jam_mulai))
                        scheduled_courses.add(matkul.kode_mk)
                else:
                    # Mata kuliah teori tanpa pasangan praktikum atau praktikumnya sudah dijadwalkan
                    ruang = self.get_ruang_valid(matkul)
                    hari = random.choice(self.daftar_hari)
                    durasi = self.jam_sks(matkul.sks, matkul.kategori)
                    max_jam_mulai = len(self.daftar_slot) - durasi
                    
                    if max_jam_mulai <= 0:
                        print(f"Matkul {matkul.matkul} ({durasi} slot) tidak muat dalam hari (slot tersedia: {len(self.daftar_slot)})")
                        continue
                        
                    jam_mulai = random.randint(0, max_jam_mulai)
                    solution.append((matkul, ruang, hari, jam_mulai))
                    scheduled_courses.add(matkul.kode_mk)
            else:
                # Mata kuliah biasa tanpa pasangan (bisa praktikum tanpa teori atau mata kuliah lainnya)
                ruang = self.get_ruang_valid(matkul)
                hari = random.choice(self.daftar_hari)
                durasi = self.jam_sks(matkul.sks, matkul.kategori)
                max_jam_mulai = len(self.daftar_slot) - durasi
                
                if max_jam_mulai <= 0:
                    print(f"Matkul {matkul.matkul} ({durasi} slot) tidak muat dalam hari (slot tersedia: {len(self.daftar_slot)})")
                    continue
                    
                jam_mulai = random.randint(0, max_jam_mulai)
                solution.append((matkul, ruang, hari, jam_mulai))
                scheduled_courses.add(matkul.kode_mk)
        
        return solution
    
    def get_neighbor(self, solution):
        neighbor = solution.copy()
        index = random.randint(0, len(neighbor) - 1)
        matkul, _, _, _ = neighbor[index]
        
        # Cek apakah mata kuliah teori ini memiliki pasangan praktikum
        if matkul.kode_pasangan:
            praktikum_course = self.find_praktikum_from_teori(matkul)
            if praktikum_course:
                # Jadwalkan keduanya berurutan (teori dulu, lalu praktikum)
                durasi_teori = self.jam_sks(matkul.sks, matkul.kategori)
                durasi_praktikum = self.jam_sks(praktikum_course.sks, praktikum_course.kategori)
                total_durasi = durasi_teori + durasi_praktikum
                
                # Pastikan ada cukup slot untuk kedua mata kuliah
                max_jam_mulai = len(self.daftar_slot) - total_durasi
                if max_jam_mulai > 0:
                    ruang_teori = self.get_ruang_valid(matkul)
                    ruang_praktikum = self.get_ruang_valid(praktikum_course)
                    hari = random.choice(self.daftar_hari)
                    jam_mulai_teori = random.randint(0, max_jam_mulai)
                    jam_mulai_praktikum = jam_mulai_teori + durasi_teori
                    
                    # Update mata kuliah teori
                    neighbor[index] = (matkul, ruang_teori, hari, jam_mulai_teori)
                    
                    # Cari dan update mata kuliah praktikum
                    for i, (mk, _, _, _) in enumerate(neighbor):
                        if mk == praktikum_course:
                            neighbor[i] = (praktikum_course, ruang_praktikum, hari, jam_mulai_praktikum)
                            break
                else:
                    # Jika tidak cukup slot, gunakan cara biasa
                    ruang = self.get_ruang_valid(matkul)
                    hari = random.choice(self.daftar_hari)
                    jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks, matkul.kategori))
                    neighbor[index] = (matkul, ruang, hari, jam_mulai)
            else:
                # Mata kuliah teori tanpa pasangan praktikum
                ruang = self.get_ruang_valid(matkul)
                hari = random.choice(self.daftar_hari)
                jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks, matkul.kategori))
                neighbor[index] = (matkul, ruang, hari, jam_mulai)
        
        # Cek apakah mata kuliah praktikum ini memiliki pasangan teori
        elif matkul.kategori == "Praktikum":
            teori_course = self.find_teori_from_praktikum(matkul)
            if teori_course:
                # Jadwalkan keduanya berurutan (teori dulu, lalu praktikum)
                durasi_teori = self.jam_sks(teori_course.sks, teori_course.kategori)
                durasi_praktikum = self.jam_sks(matkul.sks, matkul.kategori)
                total_durasi = durasi_teori + durasi_praktikum
                
                # Pastikan ada cukup slot untuk kedua mata kuliah
                max_jam_mulai = len(self.daftar_slot) - total_durasi
                if max_jam_mulai > 0:
                    ruang_teori = self.get_ruang_valid(teori_course)
                    ruang_praktikum = self.get_ruang_valid(matkul)
                    hari = random.choice(self.daftar_hari)
                    jam_mulai_teori = random.randint(0, max_jam_mulai)
                    jam_mulai_praktikum = jam_mulai_teori + durasi_teori
                    
                    # Update mata kuliah praktikum
                    neighbor[index] = (matkul, ruang_praktikum, hari, jam_mulai_praktikum)
                    
                    # Cari dan update mata kuliah teori
                    for i, (mk, _, _, _) in enumerate(neighbor):
                        if mk == teori_course:
                            neighbor[i] = (teori_course, ruang_teori, hari, jam_mulai_teori)
                            break
                else:
                    # Jika tidak cukup slot, gunakan cara biasa
                    ruang = self.get_ruang_valid(matkul)
                    hari = random.choice(self.daftar_hari)
                    jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks, matkul.kategori))
                    neighbor[index] = (matkul, ruang, hari, jam_mulai)
            else:
                # Mata kuliah praktikum tanpa pasangan teori
                ruang = self.get_ruang_valid(matkul)
                hari = random.choice(self.daftar_hari)
                jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks, matkul.kategori))
                neighbor[index] = (matkul, ruang, hari, jam_mulai)
        else:
            # Mata kuliah biasa tanpa pasangan
            ruang = self.get_ruang_valid(matkul)
            hari = random.choice(self.daftar_hari)
            jam_mulai = random.randint(0, len(self.daftar_slot) - self.jam_sks(matkul.sks, matkul.kategori))
            neighbor[index] = (matkul, ruang, hari, jam_mulai)
        
        return neighbor
    
    def calculate_energy(self, solution):
        conflicts = 0
        
        # Existing conflict checks
        for i, (mk1, r1, h1, j1) in enumerate(solution):
            slots_needed1 = self.jam_sks(mk1.sks, mk1.kategori)
            
            if mk1.ruang_needed and not any(t in r1.tipe_ruang for t in mk1.ruang_needed):
                conflicts += 10
                
            for mk2, r2, h2, j2 in solution[i+1:]:
                slots_needed2 = self.jam_sks(mk2.sks, mk2.kategori)
                
                if h1 == h2:
                    if r1 == r2 and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
                    if mk1.dosen == mk2.dosen and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
                    if mk1.prodi == mk2.prodi and mk1.semester == mk2.semester and max(j1, j2) < min(j1 + slots_needed1, j2 + slots_needed2):
                        conflicts += 1
        
        # Tambahan: Penalti untuk mata kuliah berpasangan yang tidak berurutan
        for i, (mk1, r1, h1, j1) in enumerate(solution):
            # Jika mk1 adalah mata kuliah teori yang memiliki pasangan praktikum
            if mk1.kode_pasangan:
                praktikum_course = self.find_praktikum_from_teori(mk1)
                if praktikum_course:
                    # Cari jadwal mata kuliah praktikum
                    for mk2, r2, h2, j2 in solution:
                        if mk2 == praktikum_course:
                            slots_needed1 = self.jam_sks(mk1.sks, mk1.kategori)
                            
                            # Penalti jika tidak di hari yang sama
                            if h1 != h2:
                                conflicts += 5
                            # Penalti jika praktikum tidak langsung setelah teori
                            elif j1 + slots_needed1 != j2:
                                conflicts += 3
                            break
            
            # Atau jika mk1 adalah mata kuliah praktikum yang memiliki pasangan teori
            elif mk1.kategori == "Praktikum":
                teori_course = self.find_teori_from_praktikum(mk1)
                if teori_course:
                    # Cari jadwal mata kuliah teori
                    for mk2, r2, h2, j2 in solution:
                        if mk2 == teori_course:
                            slots_needed2 = self.jam_sks(mk2.sks, mk2.kategori)
                            
                            # Penalti jika tidak di hari yang sama
                            if h1 != h2:
                                conflicts += 5
                            # Penalti jika praktikum tidak langsung setelah teori
                            elif j2 + slots_needed2 != j1:
                                conflicts += 3
                            break
        
        return conflicts
    
    def accept_probability(self, current_energy, neighbor_energy, temperature):
        if neighbor_energy < current_energy:
            return 1.0
        return math.exp((current_energy - neighbor_energy) / temperature)
    
    def apply_solution(self, solution):
        self.reset_jadwal()
        for matkul, ruang, hari, jam_mulai in solution:
            slots_needed = self.jam_sks(matkul.sks,matkul.kategori)
            for i in range(slots_needed):
                if jam_mulai + i < len(self.daftar_slot):  # pastikan jam_list
                    ruang.jadwal[hari][jam_mulai + i] = matkul
                    matkul.dosen.jadwal[hari][jam_mulai + i] = matkul
                    self.prodi_jadwal[matkul.prodi][matkul.semester][hari][jam_mulai + i] = matkul

    def reset_jadwal(self):
        for ruang in self.daftar_ruang:
            ruang.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        for dosen in self.daftar_dosen:
            dosen.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
    
    def generate_jadwal(self):
        self.anneal()
        return self.best_solution
    
    def tampilkan_slot_waktu(self):
        print("Daftar Slot Waktu:")
        for i, slot in enumerate(self.daftar_slot, 1):
            print(f"{i}. {slot[0]} - {slot[1]}")

    def simpan_optimasi(self, df, table_name='tb_hasil'):
        df.to_sql(table_name, con=self.engine, if_exists='append', index=False)
        print(f'Data berhasil disimpan ke database {table_name}')
        with self.engine.begin() as conn:
            query = text("UPDATE tb_generate SET status = :status WHERE id_generate = :id_generate")
            conn.execute(query, {"status": "sudah", "id_generate": self.id_generate})
            
    def df_hasiljadwal(self, solution):
        data=[]
        for matkul, ruang, hari, jam_mulai in solution:
            butuh_slot = self.jam_sks(matkul.sks, matkul.kategori)

            waktu_mulai = self.daftar_slot[jam_mulai][0]
            waktu_selesai = self.daftar_slot[jam_mulai + butuh_slot - 1][1]         

            data.append({
                'id_perkuliahan': matkul.id_perkuliahan,
                'hari': hari,
                'jam_mulai': waktu_mulai,
                'jam_selesai': waktu_selesai,
                'kelas': matkul.kelas,
                'mata_kuliah': matkul.matkul,
                'nama_dosen': matkul.dosen.nama,
                'ruang': ruang.nama if hasattr(ruang, 'nama') else ruang,
                'semester': matkul.semester
            })
        df = pd.DataFrame(data)
        return df
    
sa = PenjadwalanSA()
best_solution, best_score = sa.anneal()
print("Skor akhir:", best_score)
print("Jumlah jadwal gagal:", sa.jumlah_jadwal_gagal(best_solution), "dari total", len(sa.daftar_matkul))

sa.tampilkan_jadwal(best_solution)
df_jadwal = sa.df_jadwaloptimasi(best_solution) #pastikan df_jadwal memiliki id_perkuliahan

# sa.simpan_optimasi(df_jadwal)


TypeError: Matakuliah.__init__() takes from 10 to 11 positional arguments but 13 were given

In [ ]:
from sqlalchemy import text
import requests
import random
from datetime import datetime, timedelta
import time
import math
from collections import defaultdict
import pandas as pd
from config import db_url

class Ruang:
    def __init__(self, nama, tipe_ruang):
        self.nama = nama
        self.tipe_ruang = tipe_ruang
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        self.kapasitas = 40  # Default kapasitas, bisa disesuaikan
    
    def __repr__(self):
        return f"Ruang(nama={self.nama}, tipe_ruang={self.tipe_ruang})"
    
    def __eq__(self, other):
        return isinstance(other, Ruang) and self.nama == other.nama
    
    def __hash__(self):
        return hash(self.nama)

class Dosen:
    def __init__(self, nama):
        self.nama = nama
        self.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        self.preferensi_hari = []  # Bisa ditambahkan preferensi hari
    
    def __repr__(self):
        return f"Dosen(nama={self.nama})"
    
    def __eq__(self, other):
        return isinstance(other, Dosen) and self.nama == other.nama
    
    def __hash__(self):
        return hash(self.nama)

class Matakuliah:
    def __init__(self, matkul, dosen, sks, kelas, id_perkuliahan, id_semester, semester, kategori, prodi, status=None):
        self.id_perkuliahan = id_perkuliahan
        self.matkul = matkul
        self.dosen = dosen
        self.sks = sks
        self.status = status
        self.kelas = kelas
        self.id_semester = id_semester
        self.semester = semester
        self.kategori = kategori
        self.prodi = prodi
        self.ruang_needed = self.set_ruang(kategori, status)
        self.prioritas = self.set_prioritas()

    def __repr__(self):
        return f"Matakuliah(matkul={self.matkul}, dosen={self.dosen.nama}, sks={self.sks}, kelas={self.kelas})"
    
    def set_ruang(self, kategori, status=None):
        if kategori == "Teori":
            return [status] if status else ["Kelas"]
        elif kategori == "Praktikum":
            return ["Lab"]
        elif kategori == "Gabungan":
            return [status] if status else ["Kelas", "Lab"]
        else:
            return ["Kelas"]  # Default fallback
    
    def set_prioritas(self):
        """Menentukan prioritas berdasarkan kategori dan SKS"""
        if self.kategori == "Praktikum":
            return 3  # Prioritas tinggi karena butuh lab khusus
        elif self.sks >= 3:
            return 2  # Prioritas sedang untuk mata kuliah dengan SKS tinggi
        else:
            return 1  # Prioritas rendah

class PenjadwalanSA:
    def __init__(self, initial_temperature=1000, cooling_rate=0.995, max_iterations=100000, id_generate=1):
        self.initial_temperature = initial_temperature
        self.cooling_rate = cooling_rate
        self.max_iterations = max_iterations
        self.id_generate = id_generate
        self.engine = db_url
        
        # Data penjadwalan
        self.daftar_dosen = []
        self.daftar_ruang = []
        self.daftar_matkul = []
        self.daftar_hari = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']
        
        # Slot waktu 
        self.durasi_slot = timedelta(minutes=50)
        self.jam_mulai = datetime.strptime("07:00", "%H:%M")
        self.jam_selesai = datetime.strptime("19:05", "%H:%M")
        self.slot_istirahat = [
            (datetime.strptime("09:30", "%H:%M"), datetime.strptime("09:45", "%H:%M")),
            (datetime.strptime("12:15", "%H:%M"), datetime.strptime("12:45", "%H:%M")),
            (datetime.strptime("15:15", "%H:%M"), datetime.strptime("15:30", "%H:%M")),            
            (datetime.strptime("18:00", "%H:%M"), datetime.strptime("18:15", "%H:%M")),
        ]
        self.daftar_slot = self.generate_slot_waktu()
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
        
        # Tracking untuk optimasi
        self.best_solution = None
        self.best_energy = float('inf')
        self.energy_history = []
        
        # Load data
        self.baca_datamk()
        self.baca_dataruang()

    def baca_datamk(self):
        query = text("""
        SELECT tb_rombel.nama_kelas AS kelas,
            tb_matakuliah.nama_matakuliah AS matakuliah,
            tb_dosen.nama_dosen AS dosen,
            tb_matakuliah.sks AS sks,
            tb_matakuliah.status AS status,
            tb_matakuliah.kategori AS kategori,
            id_perkuliahan,
            id_semester,
            tb_matakuliah.nama_semester AS semester,
            tb_prodi.nama_prodi AS prodi
        FROM tb_perkuliahan                    
        JOIN tb_matakuliah ON tb_perkuliahan.kode_matakuliah = tb_matakuliah.kode_matakuliah
        JOIN tb_rombel ON tb_perkuliahan.id_kelasrombel = tb_rombel.id_kelasrombel
        JOIN tb_dosen ON tb_perkuliahan.kode_dosen = tb_dosen.kode_dosen
        JOIN tb_generate ON tb_generate.id_generate = tb_rombel.id_generate
        JOIN tb_prodi ON tb_perkuliahan.kode_prodi = tb_prodi.kode_prodi
        AND tb_generate.id_generate = tb_matakuliah.id_generate 
        AND tb_generate.id_generate = tb_dosen.id_generate
        AND tb_generate.id_generate = tb_prodi.id_generate
        WHERE tb_generate.id_generate = 'G2306-02'
        """)
        
        try:
            with self.engine.connect() as connection:
                df_matkul = pd.read_sql_query(query, connection, params={"id_generate": self.id_generate})

            # Iterasi per baris untuk menambahkan data
            for _, row in df_matkul.iterrows():
                dosen_obj = self.tambah_dosen(row['dosen'])
                self.tambah_matkul(
                    row['matakuliah'], dosen_obj, row['sks'], row['kelas'], 
                    row['status'], row['id_perkuliahan'], row['id_semester'],
                    row['semester'], row['kategori'], row['prodi']
                )
        except Exception as e:
            print(f"Error saat membaca data mata kuliah: {e}")

    def baca_dataruang(self):
        query = text("""
        SELECT nama_ruangan, 
            status_ruangan 
        FROM tb_ruang 
        JOIN tb_generate ON tb_ruang.id_generate = tb_generate.id_generate
        WHERE tb_generate.id_generate = 'G2306-02'
        """)
        
        try:
            with self.engine.connect() as connection:
                df_ruang = pd.read_sql_query(query, connection, params={"id_generate": self.id_generate})

            # Iterasi per baris untuk menambahkan data
            for _, row in df_ruang.iterrows():
                self.tambah_ruang(row['nama_ruangan'], row['status_ruangan'])
        except Exception as e:
            print(f"Error saat membaca data ruang: {e}")
            
    def tambah_dosen(self, nama):
        for d in self.daftar_dosen:
            if d.nama == nama:
                return d
        dosen = Dosen(nama)
        self.daftar_dosen.append(dosen)
        return dosen
    
    def tambah_ruang(self, nama, tiperuang):
        ruang = Ruang(nama, tiperuang)
        self.daftar_ruang.append(ruang)
        return ruang
    
    def tambah_matkul(self, matkul, dosen, sks, kelas, status, id_perkuliahan, id_semester, semester, kategori, prodi):
        matkul_obj = Matakuliah(matkul, dosen, sks, kelas, id_perkuliahan, id_semester, semester, kategori, prodi, status)
        self.daftar_matkul.append(matkul_obj)
        return matkul_obj

    def generate_slot_waktu(self):
        daftar_slot = []
        waktu_mulai = self.jam_mulai

        while waktu_mulai + self.durasi_slot <= self.jam_selesai:
            waktu_selesai = waktu_mulai + self.durasi_slot

            konflik_istirahat = False
            for istirahat_mulai, istirahat_selesai in self.slot_istirahat:
                if not (waktu_selesai <= istirahat_mulai or waktu_mulai >= istirahat_selesai):
                    waktu_mulai = istirahat_selesai
                    konflik_istirahat = True
                    break

            if not konflik_istirahat:
                daftar_slot.append((
                    waktu_mulai.strftime("%H:%M"),
                    waktu_selesai.strftime("%H:%M")
                ))
                waktu_mulai = waktu_selesai
        return daftar_slot

    def jam_sks(self, sks, kategori):
        """Menghitung jumlah slot yang dibutuhkan berdasarkan SKS dan kategori"""
        if kategori == "Teori":
            menit_total = sks * 50
        elif kategori == "Praktikum":
            menit_total = sks * 150  # Praktikum lebih lama
        elif kategori == "Gabungan":
            menit_total = 350  # 7 slot untuk gabungan
        else:
            menit_total = sks * 50  # Default
        return math.ceil(menit_total / 50)
    
    def get_ruang_valid(self, matkul):
        """Mendapatkan ruang yang valid untuk mata kuliah"""
        ruang_valid = []
        for ruang in self.daftar_ruang:
            if not matkul.ruang_needed:
                ruang_valid.append(ruang)
            else:
                # Cek apakah tipe ruang sesuai dengan kebutuhan
                if any(tipe in ruang.tipe_ruang for tipe in matkul.ruang_needed):
                    ruang_valid.append(ruang)
        
        return random.choice(ruang_valid) if ruang_valid else random.choice(self.daftar_ruang)

    def is_valid_schedule(self, matkul, ruang, hari, jam_mulai):
        """Validasi apakah jadwal tidak konflik"""
        durasi = self.jam_sks(matkul.sks, matkul.kategori)
        
        # Cek apakah jam mulai + durasi tidak melebihi slot yang tersedia
        if jam_mulai + durasi > len(self.daftar_slot):
            return False
        
        # Cek konflik ruang
        for i in range(durasi):
            if ruang.jadwal[hari][jam_mulai + i] is not None:
                return False
        
        # Cek konflik dosen
        for i in range(durasi):
            if matkul.dosen.jadwal[hari][jam_mulai + i] is not None:
                return False
        
        # Cek konflik kelas (prodi + semester)
        for i in range(durasi):
            if self.prodi_jadwal[matkul.prodi][matkul.semester][hari][jam_mulai + i] is not None:
                return False
        
        return True

    def solusi_awal(self):
        """Membuat solusi awal dengan pendekatan greedy yang lebih baik"""
        solution = []
        # Urutkan mata kuliah berdasarkan prioritas
        sorted_matkul = sorted(self.daftar_matkul, key=lambda x: x.prioritas, reverse=True)
        
        for matkul in sorted_matkul:
            placed = False
            attempts = 0
            max_attempts = 100
            
            while not placed and attempts < max_attempts:
                ruang = self.get_ruang_valid(matkul)
                hari = random.choice(self.daftar_hari)
                durasi = self.jam_sks(matkul.sks, matkul.kategori)
                
                if durasi > len(self.daftar_slot):
                    print(f"Warning: Mata kuliah {matkul.matkul} membutuhkan {durasi} slot, melebihi slot tersedia ({len(self.daftar_slot)})")
                    break
                
                max_jam_mulai = len(self.daftar_slot) - durasi
                if max_jam_mulai < 0:
                    break
                
                jam_mulai = random.randint(0, max_jam_mulai)
                
                # Temporary assignment untuk validasi
                temp_assignment = (matkul, ruang, hari, jam_mulai)
                temp_solution = solution + [temp_assignment]
                
                if self.calculate_conflicts_for_assignment(temp_assignment, solution) == 0:
                    solution.append(temp_assignment)
                    self.apply_single_assignment(matkul, ruang, hari, jam_mulai)
                    placed = True
                
                attempts += 1
            
            if not placed:
                # Jika tidak bisa ditempatkan tanpa konflik, paksa tempatkan
                ruang = self.get_ruang_valid(matkul)
                hari = random.choice(self.daftar_hari)
                durasi = self.jam_sks(matkul.sks, matkul.kategori)
                max_jam_mulai = max(0, len(self.daftar_slot) - durasi)
                jam_mulai = random.randint(0, max_jam_mulai)
                solution.append((matkul, ruang, hari, jam_mulai))
        
        return solution

    def calculate_conflicts_for_assignment(self, assignment, existing_solution):
        """Menghitung konflik untuk satu assignment terhadap solusi yang sudah ada"""
        matkul, ruang, hari, jam_mulai = assignment
        conflicts = 0
        durasi = self.jam_sks(matkul.sks, matkul.kategori)
        
        for existing_matkul, existing_ruang, existing_hari, existing_jam in existing_solution:
            if hari == existing_hari:
                existing_durasi = self.jam_sks(existing_matkul.sks, existing_matkul.kategori)
                
                # Cek overlap waktu
                if max(jam_mulai, existing_jam) < min(jam_mulai + durasi, existing_jam + existing_durasi):
                    # Ada overlap waktu
                    if ruang == existing_ruang:
                        conflicts += 5  # Konflik ruang
                    if matkul.dosen == existing_matkul.dosen:
                        conflicts += 5  # Konflik dosen
                    if matkul.prodi == existing_matkul.prodi and matkul.semester == existing_matkul.semester:
                        conflicts += 3  # Konflik kelas
        
        return conflicts

    def apply_single_assignment(self, matkul, ruang, hari, jam_mulai):
        """Menerapkan satu assignment ke jadwal"""
        durasi = self.jam_sks(matkul.sks, matkul.kategori)
        for i in range(durasi):
            if jam_mulai + i < len(self.daftar_slot):
                ruang.jadwal[hari][jam_mulai + i] = matkul
                matkul.dosen.jadwal[hari][jam_mulai + i] = matkul
                self.prodi_jadwal[matkul.prodi][matkul.semester][hari][jam_mulai + i] = matkul
    
    def get_neighbor(self, solution):
        """Membuat neighbor solution dengan strategi yang lebih cerdas"""
        neighbor = solution.copy()
        
        # Pilih assignment secara random
        if not neighbor:
            return neighbor
            
        index = random.randint(0, len(neighbor) - 1)
        matkul, _, _, _ = neighbor[index]
        
        # Strategi neighbor: ubah ruang, hari, atau jam mulai
        strategy = random.choice(['ruang', 'hari', 'jam', 'all'])
        
        if strategy == 'ruang':
            ruang = self.get_ruang_valid(matkul)
            _, _, hari, jam_mulai = neighbor[index]
            neighbor[index] = (matkul, ruang, hari, jam_mulai)
        elif strategy == 'hari':
            _, ruang, _, jam_mulai = neighbor[index]
            hari = random.choice(self.daftar_hari)
            neighbor[index] = (matkul, ruang, hari, jam_mulai)
        elif strategy == 'jam':
            _, ruang, hari, _ = neighbor[index]
            durasi = self.jam_sks(matkul.sks, matkul.kategori)
            max_jam = max(0, len(self.daftar_slot) - durasi)
            jam_mulai = random.randint(0, max_jam)
            neighbor[index] = (matkul, ruang, hari, jam_mulai)
        else:  # all
            ruang = self.get_ruang_valid(matkul)
            hari = random.choice(self.daftar_hari)
            durasi = self.jam_sks(matkul.sks, matkul.kategori)
            max_jam = max(0, len(self.daftar_slot) - durasi)
            jam_mulai = random.randint(0, max_jam)
            neighbor[index] = (matkul, ruang, hari, jam_mulai)
        
        return neighbor
    
    def calculate_energy(self, solution):
        """Menghitung total energy (konflik) dari solusi"""
        conflicts = 0
        
        for i, (mk1, r1, h1, j1) in enumerate(solution):
            durasi1 = self.jam_sks(mk1.sks, mk1.kategori)
            
            # Penalti untuk ruang yang tidak sesuai
            if mk1.ruang_needed and not any(t in r1.tipe_ruang for t in mk1.ruang_needed):
                conflicts += 10
            
            # Penalti untuk jadwal di luar jam kerja
            if j1 + durasi1 > len(self.daftar_slot):
                conflicts += 20
            
            # Cek konflik dengan mata kuliah lain
            for j, (mk2, r2, h2, j2) in enumerate(solution[i+1:], i+1):
                if h1 == h2:  # Hari yang sama
                    durasi2 = self.jam_sks(mk2.sks, mk2.kategori)
                    
                    # Cek overlap waktu
                    if max(j1, j2) < min(j1 + durasi1, j2 + durasi2):
                        # Konflik ruang
                        if r1 == r2:
                            conflicts += 5
                        
                        # Konflik dosen
                        if mk1.dosen == mk2.dosen:
                            conflicts += 5
                        
                        # Konflik kelas (prodi + semester)
                        if mk1.prodi == mk2.prodi and mk1.semester == mk2.semester:
                            conflicts += 3
        
        return conflicts
    
    def accept_probability(self, current_energy, neighbor_energy, temperature):
        """Menghitung probabilitas penerimaan solusi"""
        if neighbor_energy < current_energy:
            return 1.0
        if temperature <= 0:
            return 0.0
        return math.exp((current_energy - neighbor_energy) / temperature)
    
    def anneal(self):
        """Algoritma Simulated Annealing utama"""
        print("Memulai Simulated Annealing...")
        
        current_solution = self.solusi_awal()
        current_energy = self.calculate_energy(current_solution)
        
        self.best_solution = current_solution.copy()
        self.best_energy = current_energy
        
        temperature = self.initial_temperature
        
        accepted_moves = 0
        total_moves = 0
        
        for iteration in range(self.max_iterations):
            neighbor_solution = self.get_neighbor(current_solution)
            neighbor_energy = self.calculate_energy(neighbor_solution)
            
            total_moves += 1
            
            # Decide whether to accept the neighbor
            if self.accept_probability(current_energy, neighbor_energy, temperature) > random.random():
                current_solution = neighbor_solution
                current_energy = neighbor_energy
                accepted_moves += 1
            
            # Update best solution
            if current_energy < self.best_energy:
                self.best_solution = current_solution.copy()
                self.best_energy = current_energy
            
            # Cool down
            temperature *= self.cooling_rate
            
            # Track energy history
            if iteration % 1000 == 0:
                self.energy_history.append(current_energy)
            
            # Progress report
            if iteration % 10000 == 0:
                acceptance_rate = accepted_moves / total_moves if total_moves > 0 else 0
                print(f"Iterasi {iteration:,} | "
                      f"Skor saat ini: {current_energy} | "
                      f"Skor terbaik: {self.best_energy} | "
                      f"Temperatur: {temperature:.4f} | "
                      f"Acceptance rate: {acceptance_rate:.3f}")
                
                # Reset counters
                accepted_moves = 0
                total_moves = 0
            
            # Early stopping jika sudah optimal
            if self.best_energy == 0:
                print(f"Solusi optimal ditemukan pada iterasi {iteration}")
                break
        
        print(f"Simulated Annealing selesai. Skor akhir: {self.best_energy}")
        self.apply_solution(self.best_solution)
        return self.best_solution

    def apply_solution(self, solution):
        """Menerapkan solusi ke jadwal"""
        self.reset_jadwal()
        for matkul, ruang, hari, jam_mulai in solution:
            self.apply_single_assignment(matkul, ruang, hari, jam_mulai)

    def reset_jadwal(self):
        """Reset semua jadwal"""
        for ruang in self.daftar_ruang:
            ruang.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        for dosen in self.daftar_dosen:
            dosen.jadwal = defaultdict(lambda: defaultdict(lambda: None))
        self.prodi_jadwal = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
    
    def generate_jadwal(self):
        """Generate jadwal dan return hasil terbaik"""
        return self.anneal()
    
    def validate_solution(self, solution):
        """Validasi solusi final"""
        conflicts = {
            'ruang': 0,
            'dosen': 0,
            'kelas': 0,
            'ruang_tidak_sesuai': 0,
            'waktu_invalid': 0
        }
        
        for i, (mk1, r1, h1, j1) in enumerate(solution):
            durasi1 = self.jam_sks(mk1.sks, mk1.kategori)
            
            # Cek ruang sesuai
            if mk1.ruang_needed and not any(t in r1.tipe_ruang for t in mk1.ruang_needed):
                conflicts['ruang_tidak_sesuai'] += 1
            
            # Cek waktu valid
            if j1 + durasi1 > len(self.daftar_slot):
                conflicts['waktu_invalid'] += 1
            
            # Cek konflik dengan mata kuliah lain
            for mk2, r2, h2, j2 in solution[i+1:]:
                if h1 == h2:
                    durasi2 = self.jam_sks(mk2.sks, mk2.kategori)
                    if max(j1, j2) < min(j1 + durasi1, j2 + durasi2):
                        if r1 == r2:
                            conflicts['ruang'] += 1
                        if mk1.dosen == mk2.dosen:
                            conflicts['dosen'] += 1
                        if mk1.prodi == mk2.prodi and mk1.semester == mk2.semester:
                            conflicts['kelas'] += 1
        
        return conflicts
    
    def tampilkan_slot_waktu(self):
        """Menampilkan daftar slot waktu"""
        print("Daftar Slot Waktu:")
        for i, slot in enumerate(self.daftar_slot, 1):
            print(f"{i:2d}. {slot[0]} - {slot[1]}")

    def simpan_optimasi(self, df, table_name='tb_hasil'):
        """Simpan hasil optimasi ke database"""
        try:
            df.to_sql(table_name, con=self.engine, if_exists='append', index=False)
            print(f'Data berhasil disimpan ke database {table_name}')
            
            with self.engine.begin() as conn:
                query = text("UPDATE tb_generate SET status = :status WHERE id_generate = :id_generate")
                conn.execute(query, {"status": "sudah", "id_generate": self.id_generate})
                print(f'Status generate ID {self.id_generate} berhasil diupdate')
        except Exception as e:
            print(f"Error saat menyimpan data: {e}")
            
    def df_hasiljadwal(self, solution):
        """Konversi solution ke DataFrame"""
        data = []
        for matkul, ruang, hari, jam_mulai in solution:
            butuh_slot = self.jam_sks(matkul.sks, matkul.kategori)
            
            if jam_mulai + butuh_slot <= len(self.daftar_slot):
                waktu_mulai = self.daftar_slot[jam_mulai][0]
                waktu_selesai = self.daftar_slot[jam_mulai + butuh_slot - 1][1]
            else:
                # Handle case where schedule exceeds available slots
                waktu_mulai = self.daftar_slot[jam_mulai][0] if jam_mulai < len(self.daftar_slot) else "Invalid"
                waktu_selesai = "Invalid"
            
            data.append({
                'id_perkuliahan': matkul.id_perkuliahan,
                'hari': hari,
                'jam_mulai': waktu_mulai,
                'jam_selesai': waktu_selesai,
                'kelas': matkul.kelas,
                'mata_kuliah': matkul.matkul,
                'nama_dosen': matkul.dosen.nama,
                'ruang': ruang.nama,
                'semester': matkul.semester,
                
            })
        
        return pd.DataFrame(data)
    
    def print_statistics(self):
        """Menampilkan statistik penjadwalan"""
        if self.best_solution:
            conflicts = self.validate_solution(self.best_solution)
            print("\n=== STATISTIK PENJADWALAN ===")
            print(f"Total mata kuliah: {len(self.daftar_matkul)}")
            print(f"Total dosen: {len(self.daftar_dosen)}")
            print(f"Total ruang: {len(self.daftar_ruang)}")
            print(f"Total slot waktu per hari: {len(self.daftar_slot)}")
            print(f"\nKonflik:")
            for jenis, jumlah in conflicts.items():
                print(f"  {jenis.replace('_', ' ').title()}: {jumlah}")
            print(f"\nSkor total: {self.best_energy}")
            
            if self.best_energy == 0:
                print("\n🎉 SUKSES: Tidak ada konflik dalam jadwal!")
            else:
                print(f"\n⚠️  Masih ada {self.best_energy} konflik yang perlu diselesaikan.")

# Contoh penggunaan
if __name__ == "__main__":
    # Inisialisasi dengan parameter yang sudah dioptimasi
    scheduler = PenjadwalanSA(
        initial_temperature=1000,
        cooling_rate=0.995,
        max_iterations=100000,
        id_generate='G2306-02'
    )
    
    # Generate jadwal
    print("Memulai proses penjadwalan...")
    best_solution = scheduler.generate_jadwal()
    
    
    # sa.tampilkan_jadwal()
    df_jadwal = scheduler.df_hasiljadwal(best_solution)
    print(df_jadwal)
    scheduler.simpan_optimasi(df_jadwal)
    
    # Tampilkan statistik
    scheduler.print_statistics()
    
    # Konversi ke DataFrame dan simpan
    if best_solution:
        df_hasil = scheduler.df_hasiljadwal(best_solution)
        print(f"\nHasil jadwal:\n{df_hasil.head()}")
        
        # Simpan ke database (uncomment jika ingin menyimpan)
        # scheduler.simpan_optimasi(df_hasil)
        
        # Tampilkan contoh jadwal per hari
        print("\n=== CONTOH JADWAL PER HARI ===")
        for hari in scheduler.daftar_hari:
            jadwal_hari = df_hasil[df_hasil['hari'] == hari].sort_values('jam_mulai')
            if not jadwal_hari.empty:
                print(f"\n{hari}:")
                for _, row in jadwal_hari.iterrows():
                    print(f"  {row['jam_mulai']}-{row['jam_selesai']} | "
                          f"{row['mata_kuliah']} ({row['kelas']}) | "
                          f"{row['nama_dosen']} | {row['ruang']}")
    else:
        print("Gagal menghasilkan jadwal yang valid.")
    